# Web scraping KIDFamMap 2 of 2:

## Inhibitor csv File Generation by Web Scraping and Parsing KIDFamMap Data

After running Web-scraping-KIDFamMap_1-of-2_Kinase-IDs.ipynb, we have a list of URLs to paste into our second KIDFamMap-scraping spider.

Here we define and run this spider to extract the UniProt IDs and inhibitor data for each of the 340 kinases which have inhibitor data available.

Once the data has been extracted from the website, it is cleaned and inserted into a Pandas data frame.

After that, the following steps take place:
- Using information in the data frame, we generate a column of URLs that our web app can use to display images of the inhibitors' chemical structures.
- Convert kinases and inhibitors to uppercase.
- Translate kinase names to match "Entry name" (UniProt ID) in kinase table.
- Remove duplicate rows.
- Remove unnecessary columns.
- Use this data frame to make two data frames: one listing the kinase-inhibitor pairs, and one listing each inhibitor alongside all of its information. These two data sets can then be linked via the inhibitor name in our relational database.
- Make primary key columns.
- Export as csv files.

Import packages

In [ ]:
import pandas as pd
import scrapy
from scrapy.crawler import CrawlerProcess

Make a spider

In [ ]:
# Use the scrapy.Spider class to make an inhibitor-details-scraping 
# spider

class InhibitorSpider( scrapy.Spider ):
    
    name = "inhibitor_spider"
    
    # Define the first action to take
        
    def start_requests( self ): 
        
        # Define which URLs to follow
        
        kinase_urls = ['http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ADRBK1&Query_Pid=ADRBK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=AKT1&Query_Pid=AKT1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=AKT2&Query_Pid=AKT2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=DMPK&Query_Pid=DMPK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=GRK6&Query_Pid=GRK6',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PDPK1&Query_Pid=PDPK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKACA&Query_Pid=PRKACA',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKCA&Query_Pid=PRKCA',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKCB&Query_Pid=PRKCB',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKCI&Query_Pid=PRKCI',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKCQ&Query_Pid=PRKCQ',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ROCK1&Query_Pid=ROCK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=RPS6KA1&Query_Pid=RPS6KA1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=RPS6KA5&Query_Pid=RPS6KA5',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=SGK1&Query_Pid=SGK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CAMK1D&Query_Pid=CAMK1D',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CAMK1G&Query_Pid=CAMK1G',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CAMK2A&Query_Pid=CAMK2A',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CAMK2B&Query_Pid=CAMK2B',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CAMK2D&Query_Pid=CAMK2D',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CAMK2G&Query_Pid=CAMK2G',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CAMK4&Query_Pid=CAMK4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CASK&Query_Pid=CASK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CHEK1&Query_Pid=CHEK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CHEK2&Query_Pid=CHEK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=DAPK1&Query_Pid=DAPK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=DAPK2&Query_Pid=DAPK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=DAPK3&Query_Pid=DAPK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAPKAPK2&Query_Pid=MAPKAPK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAPKAPK3&Query_Pid=MAPKAPK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MKNK2&Query_Pid=MKNK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MYLK4&Query_Pid=MYLK4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PASK&Query_Pid=PASK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PIM1&Query_Pid=PIM1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PIM2&Query_Pid=PIM2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKAA2&Query_Pid=PRKAA2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STK11&Query_Pid=STK11',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STK17B&Query_Pid=STK17B',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CSNK1G1&Query_Pid=CSNK1G1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CSNK1G2&Query_Pid=CSNK1G2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CSNK1G3&Query_Pid=CSNK1G3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=VRK1&Query_Pid=VRK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDK2&Query_Pid=CDK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDK5&Query_Pid=CDK5',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDK6&Query_Pid=CDK6',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDK7&Query_Pid=CDK7',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDK9&Query_Pid=CDK9',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CLK1&Query_Pid=CLK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CLK2&Query_Pid=CLK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CLK3&Query_Pid=CLK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CSNK2A1&Query_Pid=CSNK2A1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CSNK2A2&Query_Pid=CSNK2A2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=DYRK1A&Query_Pid=DYRK1A',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=DYRK2&Query_Pid=DYRK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=GSK3B&Query_Pid=GSK3B',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAPK1&Query_Pid=MAPK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAPK10&Query_Pid=MAPK10',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAPK11&Query_Pid=MAPK11',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAPK12&Query_Pid=MAPK12',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAPK14&Query_Pid=MAPK14',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAPK3&Query_Pid=MAPK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAPK8&Query_Pid=MAPK8',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAPK9&Query_Pid=MAPK9',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PCTK1&Query_Pid=PCTK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=SRPK1&Query_Pid=SRPK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=SRPK2&Query_Pid=SRPK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=AURKA&Query_Pid=AURKA',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=BUB1&Query_Pid=BUB1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CAMKK2&Query_Pid=CAMKK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=EIF2AK2&Query_Pid=EIF2AK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ERN1&Query_Pid=ERN1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=GSG2&Query_Pid=GSG2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=NEK2&Query_Pid=NEK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=NEK7&Query_Pid=NEK7',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PLK1&Query_Pid=PLK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PLK4&Query_Pid=PLK4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STK16&Query_Pid=STK16',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=TTK&Query_Pid=TTK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=WEE1&Query_Pid=WEE1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAP2K1&Query_Pid=MAP2K1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAP2K2&Query_Pid=MAP2K2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAP2K4&Query_Pid=MAP2K4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAP2K6&Query_Pid=MAP2K6',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MST4&Query_Pid=MST4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=OXSR1&Query_Pid=OXSR1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PAK1&Query_Pid=PAK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PAK4&Query_Pid=PAK4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PAK7&Query_Pid=PAK7',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=SLK&Query_Pid=SLK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STK10&Query_Pid=STK10',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STK24&Query_Pid=STK24',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STK25&Query_Pid=STK25',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STRADA&Query_Pid=STRADA',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=TNIK&Query_Pid=TNIK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ABL1&Query_Pid=ABL1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ABL2&Query_Pid=ABL2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ALK&Query_Pid=ALK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=BTK&Query_Pid=BTK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CSF1R&Query_Pid=CSF1R',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CSK&Query_Pid=CSK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=EGFR&Query_Pid=EGFR',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=EPHA2&Query_Pid=EPHA2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=EPHA3&Query_Pid=EPHA3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=EPHA7&Query_Pid=EPHA7',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=EPHB4&Query_Pid=EPHB4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ERBB2&Query_Pid=ERBB2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ERBB3&Query_Pid=ERBB3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ERBB4&Query_Pid=ERBB4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=FES&Query_Pid=FES',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=FGFR1&Query_Pid=FGFR1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=FGFR2&Query_Pid=FGFR2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=FLT1&Query_Pid=FLT1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=FYN&Query_Pid=FYN',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=HCK&Query_Pid=HCK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=IGF1R&Query_Pid=IGF1R',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=INSR&Query_Pid=INSR',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ITK&Query_Pid=ITK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=JAK1&Query_Pid=JAK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=JAK2&Query_Pid=JAK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=JAK3&Query_Pid=JAK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=KDR&Query_Pid=KDR',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=KIT&Query_Pid=KIT',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=LCK&Query_Pid=LCK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=LYN&Query_Pid=LYN',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MERTK&Query_Pid=MERTK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MET&Query_Pid=MET',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MST1R&Query_Pid=MST1R',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PTK2&Query_Pid=PTK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PTK2B&Query_Pid=PTK2B',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=RET&Query_Pid=RET',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=SRC&Query_Pid=SRC',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=SYK&Query_Pid=SYK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=TEK&Query_Pid=TEK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=TNK2&Query_Pid=TNK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=TYK2&Query_Pid=TYK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ZAP70&Query_Pid=ZAP70',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ACVR1&Query_Pid=ACVR1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ACVR2A&Query_Pid=ACVR2A',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ACVRL1&Query_Pid=ACVRL1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=BMPR1B&Query_Pid=BMPR1B',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=BMPR2&Query_Pid=BMPR2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=BRAF&Query_Pid=BRAF',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ILK&Query_Pid=ILK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=IRAK4&Query_Pid=IRAK4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=KSR2&Query_Pid=KSR2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAP3K7&Query_Pid=MAP3K7',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAP3K9&Query_Pid=MAP3K9',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=RAF1&Query_Pid=RAF1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=TGFBR1&Query_Pid=TGFBR1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ADRBK2&Query_Pid=ADRBK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=AKT3&Query_Pid=AKT3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDC42BPA&Query_Pid=CDC42BPA',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDC42BPB&Query_Pid=CDC42BPB',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDC42BPG&Query_Pid=CDC42BPG',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CIT&Query_Pid=CIT',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=FLJ25006&Query_Pid=FLJ25006',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=GRK1&Query_Pid=GRK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=GRK4&Query_Pid=GRK4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=GRK5&Query_Pid=GRK5',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=GRK7&Query_Pid=GRK7',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=LOC375449&Query_Pid=LOC375449',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAST1&Query_Pid=MAST1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAST2&Query_Pid=MAST2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAST3&Query_Pid=MAST3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PKN1&Query_Pid=PKN1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PKN2&Query_Pid=PKN2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PKN3&Query_Pid=PKN3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKACB&Query_Pid=PRKACB',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKACG&Query_Pid=PRKACG',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKCD&Query_Pid=PRKCD',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKCE&Query_Pid=PRKCE',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKCG&Query_Pid=PRKCG',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKCH&Query_Pid=PRKCH',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKCZ&Query_Pid=PRKCZ',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKG1&Query_Pid=PRKG1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKG2&Query_Pid=PRKG2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKX&Query_Pid=PRKX',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKY&Query_Pid=PRKY',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ROCK2&Query_Pid=ROCK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=RPS6KA2&Query_Pid=RPS6KA2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=RPS6KA3&Query_Pid=RPS6KA3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=RPS6KA4&Query_Pid=RPS6KA4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=RPS6KA6&Query_Pid=RPS6KA6',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=SGK2&Query_Pid=SGK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=SGK3&Query_Pid=SGK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STK32A&Query_Pid=STK32A',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STK32B&Query_Pid=STK32B',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STK32C&Query_Pid=STK32C',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STK38&Query_Pid=STK38',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STK38L&Query_Pid=STK38L',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CAMK1&Query_Pid=CAMK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CAMKV&Query_Pid=CAMKV',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=DCLK1&Query_Pid=DCLK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=DCLK2&Query_Pid=DCLK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=DCLK3&Query_Pid=DCLK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=KALRN&Query_Pid=KALRN',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=LOC729871&Query_Pid=LOC729871',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MELK&Query_Pid=MELK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MYLK&Query_Pid=MYLK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MYLK2&Query_Pid=MYLK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MYLK3&Query_Pid=MYLK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=OBSCN&Query_Pid=OBSCN',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PIM3&Query_Pid=PIM3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PNCK&Query_Pid=PNCK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKAA1&Query_Pid=PRKAA1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKD1&Query_Pid=PRKD1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKD2&Query_Pid=PRKD2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRKD3&Query_Pid=PRKD3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PSKH1&Query_Pid=PSKH1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PSKH2&Query_Pid=PSKH2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STK17A&Query_Pid=STK17A',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STK33&Query_Pid=STK33',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=TRIO&Query_Pid=TRIO',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CSNK1A1&Query_Pid=CSNK1A1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CSNK1A1L&Query_Pid=CSNK1A1L',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CSNK1D&Query_Pid=CSNK1D',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CSNK1E&Query_Pid=CSNK1E',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CCRK&Query_Pid=CCRK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDC2&Query_Pid=CDC2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDC2L5&Query_Pid=CDC2L5',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDC2L6&Query_Pid=CDC2L6',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDK10&Query_Pid=CDK10',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDK11A&Query_Pid=CDK11A',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDK15&Query_Pid=CDK15',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDK3&Query_Pid=CDK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDK8&Query_Pid=CDK8',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDKL1&Query_Pid=CDKL1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDKL2&Query_Pid=CDKL2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDKL3&Query_Pid=CDKL3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDKL4&Query_Pid=CDKL4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CDKL5&Query_Pid=CDKL5',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CLK4&Query_Pid=CLK4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CRKRS&Query_Pid=CRKRS',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=DYRK1B&Query_Pid=DYRK1B',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=DYRK3&Query_Pid=DYRK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=DYRK4&Query_Pid=DYRK4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=GSK3A&Query_Pid=GSK3A',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=HIPK1&Query_Pid=HIPK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=HIPK2&Query_Pid=HIPK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=HIPK3&Query_Pid=HIPK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=HIPK4&Query_Pid=HIPK4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ICK&Query_Pid=ICK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAK&Query_Pid=MAK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAPK15&Query_Pid=MAPK15',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAPK7&Query_Pid=MAPK7',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=NLK&Query_Pid=NLK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PCTK2&Query_Pid=PCTK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PCTK3&Query_Pid=PCTK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PFTK1&Query_Pid=PFTK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PRPF4B&Query_Pid=PRPF4B',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=RAGE&Query_Pid=RAGE',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=SRPK3&Query_Pid=SRPK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=AURKB&Query_Pid=AURKB',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=AURKC&Query_Pid=AURKC',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CAMKK1&Query_Pid=CAMKK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ERN2&Query_Pid=ERN2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=NEK1&Query_Pid=NEK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=NEK10&Query_Pid=NEK10',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=NEK11&Query_Pid=NEK11',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=NEK3&Query_Pid=NEK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=NEK4&Query_Pid=NEK4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=NEK5&Query_Pid=NEK5',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=NEK6&Query_Pid=NEK6',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=NEK8&Query_Pid=NEK8',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=NEK9&Query_Pid=NEK9',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PLK2&Query_Pid=PLK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PLK3&Query_Pid=PLK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STK36&Query_Pid=STK36',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ULK2&Query_Pid=ULK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ULK3&Query_Pid=ULK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=WEE2&Query_Pid=WEE2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ALS2CR2&Query_Pid=ALS2CR2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAP2K3&Query_Pid=MAP2K3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAP2K5&Query_Pid=MAP2K5',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAP4K3&Query_Pid=MAP4K3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAP4K4&Query_Pid=MAP4K4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAP4K5&Query_Pid=MAP4K5',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MINK1&Query_Pid=MINK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MYO3A&Query_Pid=MYO3A',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MYO3B&Query_Pid=MYO3B',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=NRK&Query_Pid=NRK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PAK2&Query_Pid=PAK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PAK3&Query_Pid=PAK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STK39&Query_Pid=STK39',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=AATK&Query_Pid=AATK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=AXL&Query_Pid=AXL',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=BLK&Query_Pid=BLK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=BMX&Query_Pid=BMX',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=DDR1&Query_Pid=DDR1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=DDR2&Query_Pid=DDR2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=EPHA1&Query_Pid=EPHA1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=EPHA10&Query_Pid=EPHA10',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=EPHB1&Query_Pid=EPHB1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=EPHB2&Query_Pid=EPHB2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=EPHB3&Query_Pid=EPHB3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=EPHB6&Query_Pid=EPHB6',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=FER&Query_Pid=FER',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=FGFR3&Query_Pid=FGFR3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=FGFR4&Query_Pid=FGFR4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=FGR&Query_Pid=FGR',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=FLT4&Query_Pid=FLT4',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=FRK&Query_Pid=FRK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=INSRR&Query_Pid=INSRR',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=LMTK3&Query_Pid=LMTK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=LTK&Query_Pid=LTK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MATK&Query_Pid=MATK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MUSK&Query_Pid=MUSK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=NTRK1&Query_Pid=NTRK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=NTRK2&Query_Pid=NTRK2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=NTRK3&Query_Pid=NTRK3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PDGFRA&Query_Pid=PDGFRA',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PDGFRB&Query_Pid=PDGFRB',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PTK6&Query_Pid=PTK6',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=PTK7&Query_Pid=PTK7',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ROR1&Query_Pid=ROR1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ROR2&Query_Pid=ROR2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ROS1&Query_Pid=ROS1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=RYK&Query_Pid=RYK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=SRMS&Query_Pid=SRMS',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STYK1&Query_Pid=STYK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=TEC&Query_Pid=TEC',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=TIE1&Query_Pid=TIE1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=TNK1&Query_Pid=TNK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=TXK&Query_Pid=TXK',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=TYRO3&Query_Pid=TYRO3',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=YES1&Query_Pid=YES1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ACVR1B&Query_Pid=ACVR1B',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ACVR1C&Query_Pid=ACVR1C',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=AMHR2&Query_Pid=AMHR2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ARAF&Query_Pid=ARAF',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=BMPR1A&Query_Pid=BMPR1A',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=IRAK1&Query_Pid=IRAK1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=KIAA1804&Query_Pid=KIAA1804',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=KSR1&Query_Pid=KSR1',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAP3K10&Query_Pid=MAP3K10',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAP3K11&Query_Pid=MAP3K11',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAP3K12&Query_Pid=MAP3K12',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAP3K13&Query_Pid=MAP3K13',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=TGFBR2&Query_Pid=TGFBR2',
                       'http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ZAK&Query_Pid=ZAK']

        # Go to the website at the above URLs and get a response object
        # which contains the HTML code for that web page
        # Define what to do with the response object
        # i.e. send it to the parse method defined below
        
        for url in kinase_urls:  
            yield scrapy.Request( url = url, callback = self.parse )      
 
    def parse( self, response ):
        
        # Each inhibitor list web page has information we'd like to
        # extract and place into a list "inhibs" 
        # (which we must initialise in the next cell rather than here)
        # We also want to extract the UniProt ID and kinase ID
        # in order to translate the latter to the former
        # for cross-referencing in our database
        # Define CSS or xpath locators to point to the required data 
        # and extract the text
       
        raw = response.css( 'div.result tbody > tr' ).extract()
        uniprot = response.xpath( '//a[@target="_blank"]/text()' ).extract()
        kin = response.css( 'td.tdleft5::text' ).extract()
        
        # Save the data in a list "fields"
        
        fields = [ field for field in raw ]
        
        # For each kinase, append the information about its
        # inhibitors to "inhibs"
        
        inhibs.append( fields )
        kins.append( kin )
        uniprots.append( uniprot )

Run the spider: crawl KIDFamMap for inhibitors

In [ ]:
inhibs = []
kins = []
uniprots = []

# Run the spider

process = CrawlerProcess()
process.crawl( InhibitorSpider )
process.start()

# N.B. kernel needs to be cleared before repeating

Clean up the data in "inhibs" and store in "inhibitors"

In [ ]:
inhibitors = []

for i in inhibs: # for each kinase
    for j in i: # for each kinase-inhibitor relationship
        chemical = []
        inh = j.split( "</td>" ) # Split row into individual fields
        for k in inh: # for each field, remove unnecessary characters
            field = k.replace( "<tr>", "" ) 
            field = field.replace( "\r", "" )
            field = field.replace( "\t", "" )
            field = field.replace( "<td>", "" )
            field = field.replace( "\n", "" )
            chemical.append( field ) # Make a row of cleaned, separate fields
        inhibitors.append( chemical ) # Add this row to "inhibitors"

Define column names, based on those on the KIDFamMap website

In [ ]:
headers = [ "Index", "Kinase", "Inhibitor", "Partial_Img_URL",
           "Ki_nM", "IC50_nM", "Kd_nM", "EC50_nM", "POC", "Source",
           "Link", "To_Remove" ]

Save the inhibitors information in a Pandas data frame

In [ ]:
inhibitors_df = pd.DataFrame( inhibitors, columns = headers )

Generate a dictionary of kinase names with their UniProt IDs

In [ ]:
kinases = []

for i in kins:
    kin = str( i )
    kin = kin.replace( "[", "" )
    kin = kin.replace( "]", "" )
    kin = kin.replace( "\\r", "" )
    kin = kin.replace( "\\t", "" )
    kin = kin.replace( "<td>", "" )
    kin = kin.replace( "\\n", "" )
    kin = kin.replace( "(', ')", "" )
    kin = kin.replace( "'", "" )
    kin = kin.replace( " ", "" )
    kin = kin.split( "," )
    kinases.append( kin[ 0 ] ) 

uniprot_ids = []

for i in uniprots:
    up = str( i )
    up = up.replace( "[", "" )
    up = up.replace( "]", "" )
    up = up.replace( "\\r", "" )
    up = up.replace( "\\t", "" )
    up = up.replace( "<td>", "" )
    up = up.replace( "\\n", "" )
    up = up.replace( "(', ')", "" )
    up = up.replace( "'", "" )
    up = up.replace( " ", "" )
    up = up.split( "," )
    uniprot_ids.append( str( up[ 0 ] ) )

In [ ]:
uniprot_dict = dict( list( zip( kinases, uniprot_ids ) ) )

Translate the kinase IDs to UniProt IDs and store in column

In [ ]:
UniProt_ID = []

for n, i in enumerate( inhibitors_df.Kinase ):
    if i in uniprot_dict.keys():
        uni = uniprot_dict.get( i )
        UniProt_ID.append( uni )
    else:
        UniProt_ID.append( float( 'NaN' ) )

UniProt_ID = pd.Series( UniProt_ID )
inhibitors_df = inhibitors_df.assign( UniProt_ID = UniProt_ID )

Using information in the data frame, generate a column of URLs for the inhibitors' chemical structure images. Our web app can subsequently use these to display images.

In [ ]:
IMG_URL = []

for n,i in enumerate( inhibitors_df.Partial_Img_URL ):
    URL = 'http://gemdock.life.nctu.edu.tw/kidfammap/data/png/'
    URL += str( inhibitors_df.Source[ n ] ) + "/"
    URL += str( i ) + ".png"
    IMG_URL.append( URL )

IMG_URL = pd.Series( IMG_URL )
inhibitors_df = inhibitors_df.assign( IMG_URL = IMG_URL )

Make "kinase" and "inhibitor" entries uppercase.

In [ ]:
uppercase_kinase = []
uppercase_inhib = []

for n,i in enumerate( inhibitors_df.Kinase ):
    uppercase_kinase.append( str( i ).upper() )
    uppercase_inhib.append( inhibitors_df.Inhibitor[ n ].upper() )

uppercase_kinase = pd.Series( uppercase_kinase )
uppercase_inhib = pd.Series( uppercase_inhib )

inhibitors_df = inhibitors_df.assign( Kinase = uppercase_kinase )
inhibitors_df = inhibitors_df.assign( Inhibitor = uppercase_inhib )

Make a temporary column combining the inhibitor and kinase names, to check for duplicates

In [ ]:
unique = []

for n,i in enumerate( inhibitors_df.Inhibitor ):
    uniq = str( i ) + "_" + str( inhibitors_df.Kinase[ n ] )
    unique.append( uniq )

unique = pd.Series( unique )
inhibitors_df = inhibitors_df.assign( UNIQUE = unique )

Drop any duplicate kinase-inhibitor pairs and reset the indices

In [ ]:
inhibitors_df = inhibitors_df.drop_duplicates( subset = "UNIQUE" )
inhibitors_df = inhibitors_df.reset_index( drop = True )

Drop any rows without a UniProt ID

In [ ]:
inhibitors_df = inhibitors_df.dropna( subset = [ "UniProt_ID" ] )
inhibitors_df = inhibitors_df.reset_index( drop = True )

Remove any columns not required for the web app

In [ ]:
inhibitors_df = inhibitors_df.drop( [ "Index", "To_Remove", "Partial_Img_URL", "Link", "UNIQUE" ], axis = 1 )

Make data frame of kinase-inhibitor pairs

In [ ]:
inhib_kin_df = inhibitors_df[ [ 'Kinase', 'Inhibitor', 'UniProt_ID' ] ]

Make data frame of inhibitors

In [ ]:
inhibitors_df = inhibitors_df.drop_duplicates( subset = "Inhibitor" )
inhibitors_df = inhibitors_df.drop( [ "Kinase" ], axis = 1 )
inhibitors_df = inhibitors_df.reset_index( drop = True )

Make a column of primary keys

In [ ]:
prim_key = []

count = 1

for i in inhibitors_df.Inhibitor:
    key = "IN" + "{:07d}".format( count )
    prim_key.append( key )
    count += 1

prim_key = pd.Series( prim_key )

inhibitors_df = inhibitors_df.assign( ID_IN = prim_key )

In [ ]:
prim_key = []

count = 1

for i in inhib_kin_df.Inhibitor:
    key = "KI" + "{:07d}".format( count )
    prim_key.append( key )
    count += 1

prim_key = pd.Series( prim_key )

inhib_kin_df = inhib_kin_df.assign( ID_KI = prim_key )

Write to csv

In [ ]:
inhib_kin_df.to_csv( "inhib_kin.csv", index = False )

In [ ]:
inhibitors_df.to_csv( "inhibitors.csv", index = False )